In [3]:
from openai import OpenAI
import os

openai_apikey = ""
with open('openaikey.txt',"r") as file:
    openai_apikey = file.readline().strip()
    os.environ["OPENAI_API_KEY"] = openai_apikey


client = OpenAI()

In [4]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]
user_prompt = "When's the next flight from new delhi to mumbai?"
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": user_prompt
    }
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)
print(response2)

ChatCompletion(id='chatcmpl-99S7zuiqPsJpgBxI64oeUNfLdzoeE', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"loc_origin":"DEL","loc_destination":"MUM"}', name='get_flight_info'), tool_calls=None))], created=1712040103, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=22, prompt_tokens=87, total_tokens=109))


In [9]:
from datetime import datetime,timedelta
import json
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

params=json.loads(response2.choices[0].message.function_call.arguments)
chosen_function=eval(response2.choices[0].message.function_call.name)
flight = chosen_function(**params)
print(type(flight))

<class 'str'>


In [10]:
print(flight)
response2.choices[0].message.function_call.name

{"loc_origin": "DEL", "loc_destination": "MUM", "datetime": "2024-04-03 10:29:56.222129", "airline": "KLM", "flight": "KL643"}


'get_flight_info'

In [18]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "user","content": user_prompt},
    {"role": "function", "name": "just_name", "content": flight}
      ],
    # Add function calling
  #  functions=function_descriptions,
 #   function_call="auto",  # specify the function call
    
)
print(response3)

ChatCompletion(id='chatcmpl-99nYDOifk9cWsbxfsJ56XWdQbmMiL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The next flight from New Delhi to Mumbai is on April 3, 2024, at 10:29 AM. The flight is operated by KLM with flight number KL643.', role='assistant', function_call=None, tool_calls=None))], created=1712122453, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=38, prompt_tokens=75, total_tokens=113))


In [14]:
print(response3.choices[0].message.content)

The next flight from New Delhi to Mumbai is KLM flight KL643 scheduled for April 2, 2024.
